In [1]:
!pip install torch==2.0.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 630.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.0.0+cu117 which is incompatible.
torchvision 0.20.1+cu121 requires torch==2.5.1, but you have torch 2.0.0+cu117 which is incompatible.


In [2]:
!pip install torchinfo

In [3]:
!pip install torchvision==0.15.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121


In [4]:
!pip install scikit-learn

In [5]:
!pip install braindecode==0.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 20.7 MB/s eta 0:00:00


In [6]:
!pip install mne

In [7]:
import numpy as np

def exponential_running_standardize(data, factor_new=0.001, init_block_size=None, eps=1e-4):
    """
    Perform exponential running standardization.

    Parameters
    ----------
    data : ndarray
        Input data for standardization
    factor_new : float
        Factor for updating running mean and std
    init_block_size : int
        Size of initial block for initialization of running stats
    eps : float
        Small constant for numerical stability

    Returns
    -------
    data_scaled : ndarray
        Standardized data
    """
    df = data.copy()

    if init_block_size is None:
        init_block_size = min(1000, df.shape[0])

    # Initialize mean and std with the first init_block_size samples
    mean = np.mean(df[:init_block_size], axis=0)
    std = np.std(df[:init_block_size], axis=0)
    std = np.maximum(std, eps)

    for i in range(df.shape[0]):
        if i < init_block_size:
            continue

        # Update mean and std recursively
        mean = (1 - factor_new) * mean + factor_new * df[i]
        std = (1 - factor_new) * std + factor_new * (df[i] - mean) ** 2
        std = np.maximum(std, eps)

        # Scale the current sample
        df[i] = (df[i] - mean) / np.sqrt(std)

    # Scale the initial block
    df[:init_block_size] = (df[:init_block_size] - mean) / np.sqrt(std)

    return df

In [9]:
###############################################################################
### (1) Enable logging ########################################################
###############################################################################
import warnings
warnings.simplefilter('ignore')

Timesegment = 10
Fs = 200

###############################################################################
### (2) Load data #############################################################
###############################################################################
# set up data path
import os
default_path = '/content/drive/MyDrive/Colab Notebooks/Data'
os.chdir(default_path)
del default_path

# load raw data and labels
import numpy as np
Final_Acc = np.zeros(2)
for temp_subNum in range(1, 3):
    print(f"\nProcessing Subject {temp_subNum}")

    from scipy import io
    raw_data = io.loadmat('sub_' + str(temp_subNum) + 'Day2_200508.mat')
    raw_labels = io.loadmat('sub_' + str(temp_subNum) + 'Day2_200508_ylabel.mat')
    FeatVect = raw_data['rawData']
    # 3-dimentiona array = [trials x channels x time-series]
    FeatVect = FeatVect[:, :, 0:(Timesegment*Fs)]
    y_labels = raw_labels['y_label']
    del raw_data, raw_labels

    ###########################################################################
    ### (3) Preprocessing #####################################################
    ###########################################################################
    from mne.filter import filter_data

    # Print filter information only once per subject
    print("\nFilter Information:")
    temp_data = FeatVect[0, :, :].transpose()
    temp_data = filter_data(temp_data.T, sfreq=Fs, l_freq=8, h_freq=13,
                          filter_length='auto', fir_design='firwin', verbose=True).T

    # Process all trials with verbose=False
    for ii in range(0, 60):
        temp_data = FeatVect[ii, :, :].transpose()
        temp_data = filter_data(temp_data.T, sfreq=Fs, l_freq=8, h_freq=13,
                              filter_length='auto', fir_design='firwin', verbose=False).T
        ExpRunStand_data = exponential_running_standardize(temp_data)
        ExpRunStand_data = ExpRunStand_data.transpose()
        FeatVect[ii, :, :] = ExpRunStand_data
        del temp_data, ExpRunStand_data

    ###########################################################################
    ### (3) Convert data to braindecode format ################################
    ###########################################################################
    X = (FeatVect).astype(np.float32)
    y = (y_labels).astype(np.int64)
    y = y[:, 0]
    del FeatVect, y_labels

    from sklearn.model_selection import RepeatedStratifiedKFold
    rskf = RepeatedStratifiedKFold(n_splits=4, n_repeats=1, random_state=36851234)
    CV_cnt = 0
    CV_Acc = np.zeros(4)

    import torch
    import torch.nn as nn

    # EEGNet 모델 정의
    class EEGNet(nn.Module):
        def __init__(self, n_classes, in_chans, input_time_length, dropout_rate=0.5):
            super(EEGNet, self).__init__()

            # Layer 1: Temporal Convolution
            self.conv1 = nn.Sequential(
                nn.Conv2d(1, 16, (1, 51), padding=(0, 25), bias=False),
                nn.BatchNorm2d(16)
            )

            # Layer 2: Spatial Convolution
            self.conv2 = nn.Sequential(
                nn.Conv2d(16, 16, (in_chans, 1), groups=16, bias=False),  # depthwise convolution
                nn.BatchNorm2d(16),
                nn.ELU(),
                nn.AvgPool2d((1, 4)),
                nn.Dropout(dropout_rate)
            )

            # Layer 3: Separable Convolution
            self.conv3 = nn.Sequential(
                nn.Conv2d(16, 16, (1, 15), padding=(0, 7), groups=16, bias=False),
                nn.Conv2d(16, 16, (1, 1), bias=False),  # 32가 아닌 16 유지
                nn.BatchNorm2d(16),  # 16으로 수정
                nn.ELU(),
                nn.AvgPool2d((1, 8)),
                nn.Dropout(dropout_rate)
            )

            # Output Layer
            self.fc = nn.Linear(16 * (input_time_length // 32), n_classes)

        def forward(self, x):
            # Reshape input to (batch_size, 1, channels, time)
            x = x.unsqueeze(1)

            # Temporal and Spatial Convolution
            x = self.conv1(x)
            x = self.conv2(x)

            # Separable Convolution
            x = self.conv3(x)

            # Flatten and Linear Layer
            x = x.view(x.size(0), -1)
            x = self.fc(x)

            return x

    cuda = torch.cuda.is_available()

    # Print model architecture only once per subject
    print("\nModel Architecture:")
    n_classes = 2
    in_chans = X.shape[1]
    input_time_length = X.shape[2]

    # Create temporary model for architecture display
    temp_model = EEGNet(
        n_classes=n_classes,
        in_chans=in_chans,
        input_time_length=input_time_length,
        dropout_rate=0.5
    )
    if cuda:
        temp_model.cuda()

    from torchinfo import summary
    dummy_input = torch.randn(1, in_chans, input_time_length)
    if cuda:
        dummy_input = dummy_input.cuda()
    summary(temp_model,
            input_data=dummy_input,
            verbose=2,
            col_names=["output_size", "num_params"],
            row_settings=["var_names"])
    del temp_model

    # Cross-validation loop
    for train_index, test_index in rskf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Convert to torch tensors
        X_train = torch.FloatTensor(X_train)
        y_train = torch.LongTensor(y_train)
        X_test = torch.FloatTensor(X_test)
        y_test = torch.LongTensor(y_test)

        # Split train into train and validation
        train_size = int(0.8 * len(X_train))
        valid_size = len(X_train) - train_size

        train_dataset = torch.utils.data.TensorDataset(X_train[:train_size], y_train[:train_size])
        valid_dataset = torch.utils.data.TensorDataset(X_train[train_size:], y_train[train_size:])
        test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=8, shuffle=False)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

        # Initialize EEGNet for training
        model = EEGNet(
            n_classes=n_classes,
            in_chans=in_chans,
            input_time_length=input_time_length,
            dropout_rate=0.5
        )

        if cuda:
            model.cuda()

        # Training
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
        criterion = torch.nn.CrossEntropyLoss()

        for epoch in range(30):
            model.train()
            for batch_X, batch_y in train_loader:
                if cuda:
                    batch_X = batch_X.cuda()
                    batch_y = batch_y.cuda()

                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                if cuda:
                    batch_X = batch_X.cuda()
                    batch_y = batch_y.cuda()
                outputs = model(batch_X)
                _, predicted = torch.max(outputs.data, 1)
                total += batch_y.size(0)
                correct += (predicted == batch_y).sum().item()

        accuracy = correct / total
        CV_Acc[CV_cnt] = 1 - accuracy
        CV_cnt += 1

        del model, optimizer, criterion

    Final_Acc[temp_subNum - 1] = 1 - sum(CV_Acc) / len(CV_Acc)
    del CV_Acc, y, X
    print(f"Subject {temp_subNum} Accuracy: {Final_Acc[temp_subNum - 1]:.4f}")

print(f"\nAverage Accuracy: {sum(Final_Acc)/len(Final_Acc):.4f}")

Final_Acc = Final_Acc*100


Processing Subject 1

Filter Information:
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 331 samples (1.655 s)


Model Architecture:
Layer (type (var_name))                  Output Shape              Param #
EEGNet (EEGNet)                          [1, 2]                    --
├─Sequential (conv1)                     [1, 16, 6, 2000]          --
│    └─0.weight                                                    ├─816
│    └─1.weight                                                    ├─16
│    └─1.bias                                              